<a href="https://colab.research.google.com/github/gt-cse-6040/topic_07_MT2_SP23_1467/blob/main/Topic%2007%20MT2%20SP23%20for%20Skills%20OH%20solution%20ex%201467.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Midterm 2, Spring 2023: Better Reads

_Version history:_
- 1.0.1 (Sun Apr 2): Corrected typo in Ex. 7 demo
- 1.0: Initial release

*All of the header information is important. Please read it.*

**Topics, number of exercises:** This problem builds on your knowledge of Numpy, pandas, database organization, graph abstractions, and basic Python (for interfacing with other Python libraries). It has **11** exercises, numbered 0 to **10**. There are **21** available points. However, to earn 100% the threshold is **12** points. (Therefore, once you hit **12** points, you can stop. There is no extra credit for exceeding this threshold.)

**Free points!** This exam includes one exercise, Exercise 3, whose points are "**free**." However, to get these points you need to read some text and _submit the notebook to the autograder at least once_.

**Exercise ordering:** Each exercise builds logically on previous exercises, but you may solve them in any order. Exercises are **not** necessarily ordered by difficulty, but higher point values usually imply more difficult tasks.

**Demo cells:** Code cells that start with the comment `### define demo inputs` will load results from prior exercises applied to the entire data set and use those to build demo inputs. These must be run for later demos to work properly but they do not affect the test cells. The data loaded by these cells may be large (at least in terms of human readability). You are free to inspect them, but we did not print them in the starter code.

**Debugging you code:** Right before each exercise test cell, there is a block of text explaining the variables available to you for debugging. You may use these to test your code and can print/display them as needed (careful when printing large objects, you may want to print the head or chunks of rows at a time).

**Exercise point breakdown:**

- Exercise 0: **2** points
- Exercise 1: **1** point
- Exercise 2: **3** points
- Exercise 3: **2** point **FREEBIE! Submit to record them**
- Exercise 4: **1** point
- Exercise 5: **2** points
- Exercise 6: **2** points
- Exercise 7: **1** point
- Exercise 8: **3** points
- Exercise 9: **2** points
- Exercise 10: **2** points

**Final reminders:**

- Submit after **every exercise**
- Review the generated grade report after you submit to see what errors were returned
- Stay calm, skip problems as needed, and take short breaks at your leisure

# Background: Better Reads #

[Goodreads](https://www.goodreads.com/) is a website devoted to curating user-generated book reviews. You'll do some elementary data-mining to uncover "communities" of users who like the same books. Such insights might help users find like-minded communities and generate better book recommendations.

**Overall workflow.** This notebook has six (6) parts with about 1-3 exercises each.
* **Part A:** Analyze user-book interactions [SQL, pandas]
* **Part B:** Power-law analysis [pandas, Numpy]
* **Part C:** Edge lists, NetworkX, and graph clusters [Python, graphs]
* **Part D:** Finding communities via graph clustering [SQL, pandas]
* **Part E:** Identifying "top reads" by community [pandas]
* **Part F:** Merging inventory metadata [pandas]

# Getting started (modules) #

Skim the code cell below and then run it. Take note of the standard preloaded modules, `numpy as np`, `pandas as pd`, and `sqlite3 as db`, any or all of which you may need to construct your solutions.

The other functions are used by our demo and testing code. You can ignore them unless an exercise asks you to do otherwise.

In [ ]:
# uncomment in Google Colab
# !python --version
!pip install dill
import dill as pickle

In [ ]:
### Global Imports
%load_ext autoreload
%autoreload 2

# Standard Python modules
import sys
import numpy as np
import pandas as pd
import sqlite3 as db
import networkx as nx

In [ ]:
### Global Imports
# Some functionality needed by the notebook and demo cells:
from pprint import pprint, pformat
import math

# === Iteration === #

def isiter(x):
    """
    Returns `True` if `x` is iterable.

    Uses the "duck typing" method described here:
    https://stackoverflow.com/questions/1952464/in-python-how-do-i-determine-if-an-object-is-iterable
    """
    try:
        iterator = iter(x)
    except TypeError:
        return False
    return True

def sample_iter(I, n=1, rng_or_seed=None, replace=False, safe=True):
    from pandas import DataFrame
    from numpy import ndarray, array

    rng = get_rng(rng_or_seed, ret_type=False)
    n_sample = min(n, len(I)) if safe else n
    sample_locs = rng.choice(range(len(I)), size=n_sample, replace=replace)
    if isinstance(I, DataFrame):
        sample = I.iloc[sample_locs]
    elif isinstance(I, ndarray) or isinstance(I, list):
        sample = I[sample_locs]
    elif isinstance(I, dict):
        sample_values = list(I.keys())[sample_locs]
        sample = {k: I[k] for k in sample_values}
    else:
        J = array(list(I))
        sample = type(I)(J[sample_locs])
    return sample

# === Messages === #

def status_msg(s, verbose=True, **kwargs):
    if verbose:
        print(s, **kwargs)

# === pandas ===

def subselect(df, col, values):
    """
    Subselects rows of a `DataFrame` where the column `col`
    contains any of the given `values`.

    If `values` is a non-iterable object _or_ a `str`,
    then this function treats it as a single value to
    find.
    """
    if not isinstance(values, str) and isiter(values):
        return df[df[col].isin(values)]
    return df[df[col] == values]

# === Input/output === #

# def text_to_file(s, basename, dirname='resource/asnlib/publicdata/', overwrite=True, verbose=True):
def text_to_file(s, basename, dirname='', overwrite=True, verbose=True):
    from os.path import isfile
    filename = f"{dirname}{basename}"
    status_msg(f"Writing string to '{filename}'...", verbose=verbose)
    if not overwrite and isfile(filename):
        status_msg(f"  ==> File exists already; skipping.", verbose=verbose)
    else:
        with open(filename, "wt") as fp:
            fp.write(s)
        status_msg(f"  ==> Done!", verbose=verbose)

# def load_text_from_file(basename, dirname='resource/asnlib/publicdata/', abort_on_error=False, verbose=False):
def load_text_from_file(basename, dirname='', abort_on_error=False, verbose=False):
    from os.path import isfile
    filename = f"{dirname}{basename}"
    status_msg(f"Loading string from '{filename}'...", verbose=verbose)
    if isfile(filename):
        try:
            with open(filename, "rt") as fp:
                s = fp.read()
            status_msg(f"  ==> Done!", verbose=verbose)
        except:
            if abort_on_error:
                raise
            else:
                status_msg(f"  ==> An error occurred.", verbose=verbose)
                s = ''
    return s

# def df_to_file(df, basename, dirname='resource/asnlib/publicdata/', overwrite=True, verbose=True):
def df_to_file(df, basename, dirname='', overwrite=True, verbose=True):
    from os.path import isfile
    from dill import dumps
    filename = f"{dirname}{basename}"
    if verbose:
        print(f"Writing `DataFrame` to '{filename}'...")
    if not overwrite and isfile(filename):
        print(f"  ==> File exists already; skipping.")
    else:
        with open(filename, "wb") as fp:
            fp.write(dumps(df))
        print(f"  ==> Done!")

# def load_df_from_file(basename, dirname='resource/asnlib/publicdata/', abort_on_error=False, verbose=False):
def load_df_from_file(basename, dirname='', abort_on_error=False, verbose=False):
    from os.path import isfile
    from dill import loads
    from pandas import DataFrame
    df = DataFrame()
    filename = f"{dirname}{basename}"
    status_msg(f"Loading `DataFrame` from '{filename}'...", verbose=verbose)
    if isfile(filename):
        try:
            with open(filename, "rb") as fp:
                df = loads(fp.read())
            status_msg(f"  ==> Done!", verbose=verbose)
        except:
            if abort_on_error:
                raise
            else:
                df = DataFrame()
                status_msg(f"  ==> An error occurred.", verbose=verbose)
    return df

# def obj_to_file(df, basename, dirname='resource/asnlib/publicdata/', overwrite=True, verbose=True):
def obj_to_file(df, basename, dirname='', overwrite=True, verbose=True):
    from os.path import isfile
    from dill import dumps
    filename = f"{dirname}{basename}"
    if verbose:
        print(f"Writing object (type `{type(df)}`) to '{filename}'...")
    if not overwrite and isfile(filename):
        print(f"  ==> File exists already; skipping.")
    else:
        with open(filename, "wb") as fp:
            fp.write(dumps(df))
        print(f"  ==> Done!")

# def load_obj_from_file(basename, dirname='resource/asnlib/publicdata/', abort_on_error=False, verbose=False):
def load_obj_from_file(basename, dirname='', abort_on_error=False, verbose=False):
    from os.path import isfile
    from dill import loads
    from pandas import DataFrame
    filename = f"{dirname}{basename}"
    status_msg(f"Loading object from '{filename}'...", verbose=verbose)
    if isfile(filename):
        try:
            with open(filename, "rb") as fp:
                df = loads(fp.read())
            status_msg(f"  ==> Done! Type: `{type(df)}`", verbose=verbose)
        except:
            if abort_on_error:
                raise
            else:
                df = DataFrame()
                status_msg(f"  ==> An error occurred.", verbose=verbose)
    else:
        df = None
    return df

# def load_table_from_db(table_name, basename, dirname="resource/asnlib/publicdata/", verbose=False):
def load_table_from_db(table_name, basename, dirname="", verbose=False):
    from sqlite3 import connect
    from pandas import read_sql
    filename = f"{dirname}{basename}"
    if verbose:
        print(f"Retrieving table `{table_name}` from SQLite3 DB `{filename}`...")
    conn = connect(f"file:{filename}?mode=ro", uri=True)
    df = read_sql(f"SELECT * FROM {table_name}", conn)
    conn.close()
    if verbose:
        print(f"... done! Found {len(df)} rows.")
    return df

# ==== RNGs ==== #

# https://stackoverflow.com/questions/279561/what-is-the-python-equivalent-of-static-variables-inside-a-function
def static_vars(**kwargs):
    def decorate(func):
        for k in kwargs:
            setattr(func, k, kwargs[k])
        return func
    return decorate

@static_vars(DEFAULT_RNG=None)
def get_rng(rng_or_seed, ret_type=True):
    """
    Returns a valid pseudorandom-number generator (RNG) object
    based on how `rng_or_seed` is set:

    - An integer: Creates a new RNG with the integer as a seed
    - An existing RNG object: Returns the same object
    - `None`: Returns a global "default" RNG, which is created
      once at module initialization time.

    If `ret_type` is set, this function also returns a descriptive
    string saying which of the above cases applies. (The intent of
    this string is for use in printing as part of debugging output.)
    """
    # Initialize static variable, DEFAULT_RNG
    from numpy.random import default_rng
    if get_rng.DEFAULT_RNG is None:
        get_rng.DEFAULT_RNG = default_rng(1_234_567_890)

    if isinstance(rng_or_seed, int):
        rng = default_rng(rng_or_seed)
        rng_type = f'`default_rng({rng_or_seed})`'
    elif rng_or_seed is None:
        rng = get_rng.DEFAULT_RNG
        rng_type = f'`DEFAULT_RNG` [{rng}]'
    else:
        rng = rng_or_seed # had better be a RNG
        rng_type = f'User-supplied [{rng}]'

    return (rng, rng_type) if ret_type else rng

# ==== Plotting ==== #
def plot_series_loglog(series, ax=None, figsize=(8, 8/16*9), **kwargs):
    from matplotlib.pyplot import figure, gca
    if ax is None:
        fig = figure(figsize=figsize)
        ax = gca()
    x = series.index
    y = series.values
    ax.loglog(x, y, '.', **kwargs)
    return ax

def display_image_from_file(filename, verbose=False):
    from IPython.display import Image
    if verbose:
        print(f"Loading image, `{filename}` ...")
    display(Image(filename))
    if verbose:
        print(f"... done! (Did it appear?)")

# ==== Graph / NetworkX interfacing ===== #

def to_nx(edge_list):
    from networkx import DiGraph
    G = DiGraph()
    G.add_weighted_edges_from(edge_list)
    return G

def graph_to_matrix(G):
    try:
        from networkx import to_scipy_sparse_array # Works in 3.0
        return to_scipy_sparse_array(G)
    except:
        pass

    try:
        from networkx import to_scipy_sparse_matrix # Works in 2.5
        return to_scipy_sparse_matrix(G)
    except:
        raise

def graph_spy(G, style='matrix', ax=None, figsize=(6.5, 6.5), **kwargs):
    from matplotlib.pyplot import figure, gca
    from networkx import spring_layout, draw_networkx_nodes, draw_networkx_edges, draw_networkx_labels

    if ax is None:
        fig = figure(figsize=figsize)
        ax = gca()

    if style == 'matrix':
        A = graph_to_matrix(G)
        ax.spy(A, **kwargs)
    else:
        pos = spring_layout(G, seed=7)

        # nodes
        draw_networkx_nodes(G, pos) #, node_size=700)

        # edges
        elarge = [(u, v) for (u, v, d) in G.edges(data=True) if d["weight"] >= 0.1]
        esmall = [(u, v) for (u, v, d) in G.edges(data=True) if d["weight"] < 0.1]
        draw_networkx_edges(G, pos, edgelist=elarge, width=2)
        draw_networkx_edges(G, pos, edgelist=esmall, width=0.5, alpha=0.5)

        # node labels
        draw_networkx_labels(G, pos, font_size=10, font_family="sans-serif")
        # edge weight labels
#        edge_labels = nx.get_edge_attributes(G, "weight")
#        nx.draw_networkx_edge_labels(G, pos, edge_labels)
    return ax

def detect_communities(G, seed=1_234):
    from networkx.algorithms.community import louvain_communities
    return louvain_communities(G, seed=seed)

def random_clusters(nc, nvc, p_intra=0.5, p_inter=0.1, rng_or_seed=None, verbose=False):
    rng = get_rng(rng_or_seed, ret_type=False)
    n = nc * nvc
    mv_intra = int(p_intra*nvc) + 1 # no. of intra-cluster edges per vertex
    mv_inter = int(p_inter*n)       # no. of inter-cluster edges per vertex

    if verbose:
        print('Constructing a vertex-clustered graph with these properties:')
        print(f'- Number of clusters: nc={nc}')
        print(f'- Vertices per cluster: nvc={nvc}')
        print(f'- Number of intra-cluster edges per vertex: {mv_intra} (p_intra={p_intra})')
        print(f'- Number of inter-cluster edges per vertex: {mv_inter} (p_inter={p_inter})')
        print(f'- RNG: {rng}')

    V = set(range(n)) # `n` vertices
    E = []
    for c in range(nc):
        V_c = set(range(c*nvc, (c+1)*nvc))
        for v in V_c:
            # Add intra-cluster edges for `v`
            N_v = sample_iter(V_c - {v}, n=mv_intra, rng_or_seed=rng)
            W_v = rng.random(size=len(N_v))
            E += [(v, u, w) for u, w in zip(N_v, W_v)]

            # Add inter-cluster edges for `v`
            X_v = sample_iter(V - V_c, n=mv_inter, rng_or_seed=rng)
            W_v = rng.random(size=len(X_v)) / 10.0 # make these edges weaker, too
            E += [(v, u, w) for u, w in zip(X_v, W_v)]
    return E

In [ ]:
# import files
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/demo_ex1.db
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/demo_ex6.obj
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/demo_ex7.db
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/ex1-is_read.df
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/ex1-is_reviewed.df
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/goodreads.db
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/tc_1
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/tc_4
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/tc_6
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/tc_7
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/ex1-rating.df
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/ex1-user_id.df
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/ex6-comms.df
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/ex7-means.df
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/ex7-sampler-input.df

!mkdir tester_fw
%cd tester_fw

!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/tester_fw/__init__.py
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/tester_fw/test_utils.py
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_1467/main/tester_fw/testers.py

%cd ..

In case it's helpful, here are the versions of Python and standard modules you are using:

In [ ]:
print("* Python version: {}".format(sys.version.replace('\n', ' ')))
print(f"* Numpy version: {np.__version__}")
print(f"* pandas version: {pd.__version__}")
print(f"* sqlite3 version: {db.version}")

## pandas versus SQL ##

The actual Goodreads data is provided via a SQLite3 database. However, only some exercises _require_ SQL; most exercises were designed with pandas in mind.

Nevertheless, even some of the pandas exercises can be solved using SQL. The cell below defines the function, `dfs_to_conn`, which can be used to create in-memory database connections. If you pass in a dictionary mapping table names to pandas `DataFrame` objects, then `dfs_to_conn` will return a `sqlite3` connection with all of the data in the `DataFrame` objects available under the names given as keys. You are also free to write to the in-memory database by creating tables, inserting, deleting, updating records, etc. Anything that SQLite3 allows should work.

**Example:**

```python
    my_df = pd.DataFrame({'A':[1,2,3], 'B': [4,5,6], 'C':['x', 'y', 'z']})
    print(my_df)
    #    A  B  C
    # 0  1  4  x
    # 1  2  5  y
    # 2  3  6  z
    conn = dfs_to_conn({'my_table': my_df})
    cur = conn.cursor()
    cur.execute('select A, B, C from my_table')
    result = cur.fetchall()
    conn.close()
    print(result) # list of tuples, each tuple is a row
    #[(1, 4, 'x'), (2, 5, 'y'), (3, 6, 'z')]
```

In [ ]:
def dfs_to_conn(conn_dfs, index=False):
    import sqlite3
    conn = sqlite3.connect(':memory:')
    for table_name, df in conn_dfs.items():
        df.to_sql(table_name, conn, if_exists='replace', index=index)
    return conn

# Goodreads Data (`grdbconn`) #

Some of the Goodreads data is stored in a SQLite3 database. The code cell below opens a read-only connection to it named **`grdbconn`**.

For now, don't worry about what's there. We will explain any tables you need in the exercises that use them.

In [ ]:
# Goodreads database connection:
# grdbconn = db.connect('file:resource/asnlib/publicdata/goodreads.db?mode=ro', uri=True)
grdbconn = db.connect('file:goodreads.db?mode=ro', uri=True)

# Part A: Analyzing user-book interactions #

> Includes Exercise 0 (2 points) and Exercise 1 (1 point).

The Goodreads dataset includes **user-book interactions.** An "user-book interaction" means the user "did something" with the book on the Goodreads website:

- _Viewed_: The user looked at a book description and saved it to their personal library.
- _Read_: The user marked the book as "read."
- _Rated_: The user gave the book a rating, from 1 to 5 stars.
- _Reviewed_: The user wrote a public review of the book on the website.

These interactions are recorded in a SQL table called `Interactions`. Let's have a quick look for one of the users whose integer ID is `840218`:

In [ ]:
pd.read_sql(r"SELECT * FROM Interactions WHERE user_id=830690", grdbconn)

Each row shows how this user interacted with some book. This user interacted with five books. However, they saved books `15396` (row 0) and `39407` (row 4) but did nothing else with them—that is, they did not read them, rate them, or review them.

They did rate books `19990`, giving it `5` stars, as well as `19989`, giving it `3` stars. They also read `19988`, but they did not rate it. They did not review any book (`is_reviewed=0`). Had they done so, `is_reviewed` would be `1`. All values are integers.

## **Ex. 1 (1 pt)**: `count_interactions_by` ##

Suppose we want to group the interactions and count the number by group. For example, we might want to know, for each unique user ID, how many interactions there are. Complete the function
```python
def count_interactions_by(col, conn):
    ...
```
so that it does the following.

**Inputs:**
- `col`: The name of a column
- `conn`: A database connection containing a table named `Interactions`

**Your task:** For each unique value in column `'col'` of the `Interactions` table, count how many interactions (rows) there are.

**Output:** Return a dataframe with two columns:
- `col`: A column with the **same name** as the given input column holding the unique values
- `'count'`: A column with the number of interactions for each unique value

Refer to the demo cell below for an example of this output.

**Additional notes and hints:** You may assume that `col` holds a valid column name. The exact order of rows and columns in your output does not matter.

**Example:**

In [ ]:
### Define demo inputs ###

demo_col_ex1 = 'user_id'
demo_conn_ex1 = db.connect(f'file:demo_ex1.db?mode=ro', uri=True)
display(pd.read_sql("SELECT * FROM Interactions", demo_conn_ex1))

Calling `count_interactions_by(demo_col_ex1, demo_conn_ex1)` should produce the following output:

|   user_id |   count |
|----------:|--------:|
|    569241 |       3 |
|    604656 |       1 |
|    607817 |       4 |

However, calling `count_interactions_by('is_read', demo_conn_ex1)` would return a two-row `DataFrame` where the count of `0` and `1` values is `4` each.

In [ ]:
### Exercise 1 solution
def count_interactions_by(col, conn):
    ### BEGIN SOLUTION
    query = f"SELECT {col}, COUNT(*) AS count FROM Interactions GROUP BY {col}"
    return pd.read_sql(query, conn)
    ### END SOLUTION

### demo function calls ###
display(count_interactions_by(demo_col_ex1, demo_conn_ex1))
display(count_interactions_by('is_read', demo_conn_ex1))

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 1. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution.
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output.

In [ ]:
### test_cell_ex1
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_1',
    'func': count_interactions_by, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'col': {
            'dtype': 'str', # data type of param.
            'check_modified': False,
        },
        'conn': {
            'dtype': 'db',
            'check_modified': False
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'df',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
#            'check_column_type': True, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b'jpS7W-CpqAQfuITMEQZL-yVXfhIaCkSaei-emnyRtrI=', path='')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

print('Passed! Please submit.')

**RUN ME:** A correct implementation of `count_interactions_by`, when run on the full Goodreads dataset for the columns `is_read`, `rating`, and `is_reviewed`, would produce the following:

In [ ]:
print(f"\n=== `count_interactions_by` on the full dataset ===\n")
for col_ in ['is_read', 'rating', 'is_reviewed']:
    display(load_df_from_file(f"ex1-{col_}.df"))

> **Aside (skip if pressed for time)**: From these results, you might observe a _hint_ at a phenomenon known as a [_monotonic behavior chain_](https://dl.acm.org/doi/10.1145/3240323.3240369): the total number of interactions > the number who read > the number who rate > the number who review. Such phenomena have been used to improve automatic generation of item recommendations.

# Part C: NetworkX 101 #

> Includes Exercise 3 (the **freebie**; 2 points).

Your next major task is to learn a little bit about **NetworkX**, a popular Python package for analyzing relational data (graphs with vertices and edges).

**You do not have to write any code in this part; just read, learn, and enjoy! However, you do need to run Exercise 3's test cell and submit the exam to get its "free" points.**

Also, do pay attention: subsequent exercises will use some of these concepts.

## **Ex. 3 (_FREE_ 2 pts)**: Edge lists ##

One way to use NetworkX is to create your graph as an **edge list**, which is a Python list of tuples. Each tuple `(u, v, w)` means that vertex `u` connects to vertex `v` and that this edge from `u` to `v` has a weight `w`.

For example, here are the first five elements of an edge list for a randomly generated graph.

In [ ]:
# demo_edge_list = cse6040.utils.random_clusters(4, 6, rng_or_seed=1_234, verbose=True)

# print(f"\nThe `demo_edge_list` has {len(demo_edge_list)} tuples. The first ten are:")
# demo_edge_list[:10]

Let's use NetworkX to draw the graph that this edge list represents.

> (We've written wrappers around the corresponding NetworkX routines to hide some unnecessary detail.)

In [ ]:
# from cse6040.utils import to_nx, graph_spy

# demo_G = to_nx(demo_edge_list) # Convert edge list to NetworkX graph
# graph_spy(demo_G, style='graph', with_labels=True, figsize=(5, 5)); # Draw a picture

In this picture, the numbered circles are vertices and arrows indicate edges that go from one vertex to another. The edges are weighted, so in the picture, edges with "large" or "heavy" weights are darker and thicker than "small" or "light" weights.

The picture shows clear structure: there appear to be four **clusters** of vertices connected by heavy edges, and between these clusters there are only light edges.

To summarize, the NetworkX concepts you just saw are:

1. To use NetworkX, we need to construct **edge lists**, which are lists of tuples, `(u, v, w)`, where each tuple represents a weighted directed edge from `u` to `v` with weight `w`.
2. Our later analysis will look for **clusters** in a given network, which we will build from our data.

When you are ready, execute the cell below and submit to be sure your free points for Exercise 3 are recorded.

In [ ]:
# ### test_cell_ex3 — a freebie ###

# pass

# print('Passed! Please submit.')

# Part D: Finding communities via graph clustering #

> Includes Exercise 4 (1 point) and Exercise 5 (2 points).

**User-user interactions.** Our dataset tells us which users have viewed, read, rated, and/or reviewed the same books. We will use this fact to "connect" users to one another in a graph, and then use NetworkX to find clusters ("communities") of users.

## **Ex. 4 (1 pt)**: `form_analysis_sample` ##

If a user gave a book a rating of 4 stars or more, that is a strong signal of interest in the book. Let's start by focusing on those interactions. Complete the following function as specified.

```python
def form_analysis_sample(conn):
    ...
```

**Inputs:** `conn` is a database connection to a database with an `Interactions` table, as used in Exercises 0 and 1.

**Your task:** Return the subset of interactions where the rating is 4 or more.

**Outputs:** Return the subset of rows of `Interactions` as a pandas `DataFrame`.

**Example.** Recall the demo `Interactions` table from Exercise 1:

In [ ]:
### Define demo inputs ###

demo_conn_ex4 = db.connect(f'file:demo_ex1.db?mode=ro', uri=True)
display(pd.read_sql("SELECT * FROM Interactions", demo_conn_ex4))

# use the naming convention `demo_<parameter name>_ex<exercise number>`
# for example if the function for exercise 3 has a parameter `df`, the demo variable should be named `demo_df_ex3`

A correct implementation of `form_analysis_sample` should return the following `DataFrame`:

|   user_id |        book_id |   is_read |   rating |   is_reviewed |
|----------:|---------------:|----------:|---------:|--------------:|
|    569241 | 47199          |         1 |        5 |             1 |
|    569241 | 47383          |         1 |        5 |             1 |
|    604656 | 2345195        |         1 |        5 |             1 |

This output includes just the three rows where `rating` is `5`.

> **Note:** Although this example does not contain ratings with the value `4`, if it did, they would be included in the output.

In [ ]:
### Exercise 4 solution
def form_analysis_sample(conn):
    ### BEGIN SOLUTION
    return pd.read_sql("SELECT * FROM Interactions WHERE rating >= 4", conn)
    ### END SOLUTION

### demo function call ###
form_analysis_sample(demo_conn_ex4)

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 4. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution.
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output.

In [ ]:
### test_cell_ex4
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_4',
    'func': form_analysis_sample, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'conn':{
            'dtype': 'db', # data type of param.
            'check_modified': False,
        }
    },
    'outputs':{
        'output_0':{
            'index':0,
            'dtype':'df',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b'jpS7W-CpqAQfuITMEQZL-yVXfhIaCkSaei-emnyRtrI=', path='')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

print('Passed! Please submit.')

# Part E: Identifying "top reads" by community #

> Includes Exercise 6 (2 points), Exercise 7 (1 point), and Exercise 8 (3 points).

The NetworkX package contains several algorithms for **detecting communities**, that is, clusters of "strongly interconnected" vertices in a graph (recall Part C).

We ran one of these algorithms on a graph formed from the user-user interactions you calculated in Part D. The algorithm grouped users (graph vertices) into clusters.

It returned these clusters as a **list of sets**, where each set is a "community" of user IDs grouped together. Since users were connected for liking the same books, it's possible users in the same community have similar tastes.

Here is the communities object that NetworkX produced for us:

In [ ]:
communities = load_obj_from_file('demo_ex6.obj')

It is a list of sets:

In [ ]:
type(communities), type(communities[0])

Here is how many communities there are:

In [ ]:
len(communities)

The sizes of the 6 communities are:

In [ ]:
[len(c) for c in communities]

Let's print the smaller two:

In [ ]:
print("Community 1:", communities[1])
print("Community 4:", communities[4])

The values you see are user IDs.

## **Ex. 6 (2 pts)**: `assign_communities` ##

To merge this data with our existing database, we need to convert the Python `communities` data structure into a `DataFrame`. Complete the function below to aid in this task:

```python
def assign_communities(communities):
    ...
```

**Inputs:** The input `communities` is a list of sets of integers, as in the previous example.

**Your task:** Convert this input into a dataframe.

**Returns:** Your function should return a `DataFrame` with these columns:
- `user_id`: A user ID (an integer).
- `comm_id`: The ID of the community it belongs to (also an integer).

The community ID is its index in `communities`. That is, community `0` is stored in `communities[0]`, community `1` is in `communities[1]`, and so on.

**Example:** Consider this set of communities:

In [ ]:
### Define demo inputs ###

demo_communities_ex6 = [{1, 3, 10, 17}, {2, 6, 13, 15}, {0, 5, 11, 16}, {9, 14}, {4, 7, 8, 12}]

A correct implementation of `assign_communities` would produce this result:

|   user_id |   comm_id |
|----------:|----------:|
|         1 |         0 |
|        10 |         0 |
|         3 |         0 |
|        17 |         0 |
|         2 |         1 |
|        13 |         1 |
|         6 |         1 |
|        15 |         1 |
|         0 |         2 |
|        16 |         2 |
|        11 |         2 |
|         5 |         2 |
|         9 |         3 |
|        14 |         3 |
|         8 |         4 |
|         4 |         4 |
|        12 |         4 |
|         7 |         4 |

In [ ]:
### Exercise 6 solution
def assign_communities(communities):
    ### BEGIN SOLUTION
    from pandas import DataFrame
    all_uids = []
    all_cids = []
    for cid, uids in enumerate(communities):
        all_uids += list(uids)
        all_cids += [cid] * len(uids)
    return DataFrame({'user_id': all_uids, 'comm_id': all_cids})
    ### END SOLUTION

### demo function call ###
assign_communities(demo_communities_ex6)

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 6. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution.
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output.

In [ ]:
### test_cell_ex6
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_6',
    'func': assign_communities, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'communities': {
            'dtype': 'list', # data type of param.
            'check_modified': True,
        }
    },
    'outputs':{
        'output_0':{
            'index': 0,
            'dtype': 'df',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b'jpS7W-CpqAQfuITMEQZL-yVXfhIaCkSaei-emnyRtrI=', path='')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

print('Passed! Please submit.')

## **Ex. 7 (1 pt)**: `means_by_community` ##

Suppose we wish to calculate means (averages) of the interaction data _by community._ Implement the function,

```python
def means_by_community(intdf, comdf):
    ...
```

to perform this task.

**Inputs:**
1. `intdf`: An interactions `DataFrame` with columns `user_id`, `book_id`, `is_read`, `rating`, and `is_reviewed`.
1. `comdf`: A communities `DataFrame` with columns `user_id` and `comm_id`.

**Your task:** Join these `DataFrames` and then return a new `DataFrame` with the mean values of the `is_read`, `rating`, and `is_reviewed` columns **by community.**

**Outputs:** Your function should return a new `DataFrame` with these columns:
1. `comm_id`: An integer community ID, one per row.
2. `is_read`, `rating`, `is_reviewed`: The mean value of each column for all rows of `intdf` for all users of the community. These should be stored as `float` values.

**Additional notes:** A user ID might not appear in both inputs. These should not be part of any means calculation.

**Example:** Consider the following two inputs:

In [ ]:
### Define demo inputs ###

demo_intdf_ex7 = load_table_from_db("Interactions", "demo_ex7.db").sort_values(by='user_id')
demo_comdf_ex7 = load_table_from_db("Communities", "demo_ex7.db").sort_values(by='user_id')

display(demo_intdf_ex7)
display(demo_comdf_ex7)

A correct implementation of `means_by_community` will return:

|   comm_id |   is_read |   rating |   is_reviewed |
|----------:|----------:|---------:|--------------:|
|         0 |         1 |      4.5 |             0 |
|         3 |         1 |      5   |             0 |
|         5 |         1 |      5   |             0 |

Observe that user `34369` does not belong to any community. Therefore, none of the final averages should be affected by that user's data.

In [ ]:
### Exercise 7 solution
def means_by_community(intdf, comdf):
    ### BEGIN SOLUTION
    VALUES = ['is_read', 'rating', 'is_reviewed']
    df = intdf.merge(comdf, on='user_id')
    df = df.groupby('comm_id')[VALUES].mean().reset_index()
    for c in VALUES:
        df[c] = df[c].astype(float) # paranoia?
    return df
    ### END SOLUTION

### demo function call ###
demo_result_ex7 = means_by_community(demo_intdf_ex7, demo_comdf_ex7)
display(demo_result_ex7)

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 7. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution.
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output.

In [ ]:
### test_cell_ex7
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_7',
    'func': means_by_community, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'intdf': {
            'dtype': 'df', # data type of param.
            'check_modified': True,
        },
        'comdf': {
            'dtype': 'df',
            'check_modified': True
        }
    },
    'outputs':{
        'output_0':{
            'index': 0,
            'dtype': 'df',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b'jpS7W-CpqAQfuITMEQZL-yVXfhIaCkSaei-emnyRtrI=', path='')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

print('Passed! Please submit.')

**RUN ME:** With a correct `means_by_community`, we can see whether the communities differ in how they read, rate, and review books. Here is what would happen if we ran on the full dataset:

In [ ]:
ex7_means = load_df_from_file('ex7-means.df')
print(f"Recall: community sizes: {[(k, len(c)) for k, c in enumerate(communities)]}")
ex7_means